<a href="https://colab.research.google.com/github/xkabot/ForecastingMethods/blob/main/CNN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Creates a connected network defining the model

In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = NN(784, 10)
x = torch.randn(64, 784) # mini batch size
print(model(x).shape)

torch.Size([64, 10])


Set the device

In [ ]:
device = torch.device('cude' if torch.cuda.is_available() else 'cpu')

Set the hyperparams

In [ ]:
input_size = 784
num_classes = 10
learning_rate = .001
batch_size = 64
num_epochs = 1

Get the datasets

In [ ]:
train_Dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_Dataset, batch_size=batch_size, shuffle=True)

test_Dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_Dataset, batch_size=batch_size, shuffle=True)


Init networks

In [ ]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

Initialize network

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train network

In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device) # 64 samples, 1 input, 28 pixels, 28 pixels -> if training with 2 possible inputs
    targets = targets.to(device=device)

    # Get to correct shape
    data = data.reshape(data.shape[0], -1) # flattens into single dimension

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()

check accuracy

In [ ]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy of training data")
  else:
    print("Checking accuracy of test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}')

  # after checking accuracy return to training
  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model )

Checking accuracy of training data
Got 56069 / 60000 with accuracy 93.44833333333334
Checking accuracy of test data
Got 9308 / 10000 with accuracy 93.08


Create simple CNN

In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channel = 1, num_classes = 10): # colors = 1 total numbers = 10
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x

In [ ]:
model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)

torch.Size([64, 10])


In [ ]:
model = CNN(1, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device) # 64 samples, 1 input, 28 pixels, 28 pixels -> if training with 2 possible inputs
    targets = targets.to(device=device)

    # Get to correct shape
    # data = data.reshape(data.shape[0], -1) # flattens into single dimension

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()

In [ ]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy of training data")
  else:
    print("Checking accuracy of test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      # x = x.reshape(x.shape[0], -1) -> already reshapes in the CNN

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}')

  # after checking accuracy return to training
  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model )

Checking accuracy of training data
Got 58073 / 60000 with accuracy 96.78833333333333
Checking accuracy of test data
Got 9686 / 10000 with accuracy 96.86
